# COMP41680 Assignment 2:Text Scraping and Classification

# Problem Statement

The objective of this assignment is to scrape a corpus of news articles from a set of
web pages, pre-process the corpus, and evaluate the performance of automated
classification of these articles in a supervised learning context. 

# Part 1. Data Collection 

1.1.Base URL :http://mlg.ucd.ie/modules/COMP41680/archive/index.html
 
This URL store the information of news articles stored month wise and for each category like Sports, Business and Technology
           

### Importing important packages required by the code

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import glob
import urllib 

Declaring url ,path1 and path2 as global variable. They can be accessed from every function. The url contains the location from where the data should be scraped out. The path1 stores the location where the article's files should be stored. The path2 stores the location where the category's file should be stored

In [2]:
url = "http://mlg.ucd.ie/modules/COMP41680/archive/"
PATH = "articles/"

Defining a generic function 'get_html()' to get the html page of the url. It can be called N number of times in the program whenever required. It takes the url link as input and returns parsed page from beautiful soup.

In [3]:
#getting content of the htmls
def get_html(url):
    re = requests.get(url)  #Now, we have a Response object called re. We can get all the information we need from this object.
    data = re.text         #Requests will automatically decode content from the server. Most unicode charsets are seamlessly decoded.
    return BeautifulSoup(data, 'html.parser')  #Beautiful Soup is a Python library for pulling data out of HTML and XML files. The parser here is html

The web_scraping() function is to return all the links of the the months present on the index page. It takes the extension and filter as input and returns list of links as output. This a generic function and can be called N number of times. The scope of this function is this program.

In [4]:
#scraping the links from the urls
def web_scraping(extension, filter):
    links = []
    html = get_html(url + extension) #calling get_html to fetch the parsed html page as return object
    for link in html.find_all("a"):   #Finding all the links in the html as 'a' stores the hyperlinks
        link_href = link.get("href").strip() #getting the href values of the links and stripping for any escape character.
        if link_href!='index.html' and link_href!='': #checking if the link is empty or index.html it should be avoided
            links.append(link_href)     #if the above conditions are met then only links are appended in the list
    #print(links)
    return links    #returning the list object 


# The Functions performed by getArticleData function:

#### The getArticleData function takes the input of the links of the months and parse to the article level and retrieves the content from each of the article.

#### The getArticleData function gets the content of the article category wise.

#### The getArticleData function stores the  in the file category wise

####  The getArticleData function generates .txt files

In [5]:
#to save the data in file category wise
def getArticleData(links):
    
    for l in links:     #for every link of the month
        url1=url+l
            ##print(url)
        
            #print(page)
        soup = get_html(url1)     #get the parsed soup of the link
            #print(soup.find_all("tr"))
        otrsoupobject = soup.find_all("tr") #find all the tr of the html as the category is stored in the tr's cstegory class
        #print(otrsoupobject[1:])
        #print('===Writing Data to Files===',otrsoupobject)
        for ele in otrsoupobject[1:]:  #for every tr tag fetch the category 
            #print(ele)
            label = ele.find(class_="category").get_text().strip() #store the category's value inn the label removing escape char

            if label != "N/A":   #only get the title and content if the category;s value is present                     
                art_link = ele.find(class_="title").a["href"].strip() #get the article link for that category
                #print(art_link)
                l = url+ art_link  #form the url of the article link fetched above

                r = urllib.request.urlopen(l).read()  #get the respons for the article link
                soup = BeautifulSoup(r, "html.parser") #parse the response of the html through Beautiful Soup
                letters = soup.find_all("p")           #find all the paragraph tags as the content are stored in it
                content = soup.find("h2").get_text()   #fetch the content values
                    #print(content)
                for ele1 in letters[:-1]:               #append all the content frtched from the p tag
                    content = content + " " + ele1.get_text()    
                fileName = "data/"+label+".txt"         #store the content of the file in the respective category of that content

                f = open(fileName, "a+", encoding="utf-8")    #write the data in the txt file with filename genreated above
                f.write(content.strip())    #strip if any escape character is present
                f.write("\n")     #append a new line after content of each file
                f.close()       #close the file once the the data is wrtten
            
            

# Saving the Data in txt File

In [6]:
links = web_scraping("index.html", "month")  #fetch the links of every month 
    #print(links)
    # Web scraping for each month
getArticleData(links)   #pass the link to the function to write appropriate data in the file.

# Loading the Data from the txt file

In [7]:

df=pd.DataFrame()
CatgoryName=['business.txt','sport.txt','technology.txt']
for filename in CatgoryName:
    name1=filename.replace(".\Data\\","")
    name1=name1.replace(".txt","")
    temp_df = pd.DataFrame()
    print( " The name of the file:",name1)
    #print(filename[:-4])
    body_list = []
    f = open(".\Data\\"+filename,"r", encoding="utf-8")
    for i in f.readlines():
        body_list.append(i.strip())
    temp_df["Article"] = body_list
    temp_df["Category"] = filename[:-4]
    df = pd.concat([df,temp_df])
    


 The name of the file: business
 The name of the file: sport
 The name of the file: technology


### The dataframe loaded from the file 

In [8]:
df.head()

,Article,Category
0,Asian quake hits European shares Asian quake h...,business
1,Barclays shares up on merger talk Barclays sha...,business
2,Bush budget seeks deep cutbacks Bush budget se...,business
3,Bush to get 'tough' on deficit Bush to get 'to...,business
4,Card fraudsters 'targeting web' Card fraudster...,business


### To display the content of the article, setting the max width of the screen

In [9]:
pd.set_option('display.max_colwidth', -1)  

#### Encode input values as an enumerated type or categorical variable

In [10]:
category_id, unique_category = pd.factorize(df["Category"])
#enumerating the category as numerical value and storing as category id
#print(category_id)
#print(unique_category)

df['category_id']=category_id
df.head()

,Article,Category,category_id
0,"Asian quake hits European shares Asian quake hits European shares Shares in Europe's leading reinsurers and travel firms have fallen as the scale of the damage wrought by tsunamis across south Asia has become apparent. More than 23,000 people have been killed following a massive underwater earthquake and many of the worst hit areas are popular tourist destinations. Reisurance firms such as Swiss Re and Munich Re lost value as investors worried about rebuilding costs. But the disaster has little impact on stock markets in the US and Asia. Currencies including the Thai baht and Indonesian rupiah weakened as analysts warned that economic growth may slow. ""It came at the worst possible time,"" said Hans Goetti, a Singapore-based fund manager. ""The impact on the tourist industry is pretty devastating, especially in Thailand."" Travel-related shares dropped in Europe, with companies such as Germany's TUI and Lufthansa and France's Club Mediterranne sliding. Insurers and reinsurance firms were also under pressure in Europe. Shares in Munich Re and Swiss Re - the world's two biggest reinsurers - both fell 1.7% as the market speculated about the cost of rebuilding in Asia. Zurich Financial, Allianz and Axa also suffered a decline in value. However, their losses were much smaller, reflecting the market's view that reinsurers were likely to pick up the bulk of the costs. Worries about the size of insurance liabilities dragged European shares down, although the impact was exacerbated by light post-Christmas trading. Germany's benchmark Dax index closed the day 16.29 points lower at 3.817.69 while France's Cac index of leading shares fell 5.07 points to 3.817.69. Investors pointed out, however, that declines probably would be industry specific, with the travel and insurance firms hit hardest. ""It's still too early for concrete damage figures,"" Swiss Re's spokesman Floiran Woest told Associated Press. ""That also has to do with the fact that the damage is very widely spread geographically."" The unfolding scale of the disaster in south Asia had little immediate impact on US shares, however. The Dow Jones index had risen 20.54 points, or 0.2%, to 10,847.66 by late morning as analsyts were cheered by more encouraging reports from retailers about post-Christmas sales. In Asian markets, adjustments were made quickly to account for lower earnings and the cost of repairs. Thai Airways shed almost 4%. The country relies on tourism for about 6% of its total economy. Singapore Airlines dropped 2.6%. About 5% of Singapore's annual gross domestic product (GDP) comes from tourism. Malaysia's budget airline, AirAsia fell 2.9%. Resort operator Tanco Holdings slumped 5%. Travel companies also took a hit, with Japan's Kinki Nippon sliding 1.5% and HIS dropping 3.3%. However, the overall impact on Asia's largest stock market, Japan's Nikkei, was slight. Shares fell just 0.03%. Concerns about the strength of economic growth going forward weighed on the currency markets. The Indonesian rupiah lost as much as 0.6% against the US dollar, before bouncing back slightly to trade at 9,300. The Thai baht lost 0.3% against the US currency, trading at 39.10. In India, where more than 2,000 people are thought to have died, the rupee shed 0.1% against the dollar Analysts said that it was difficult to predict the total cost of the disaster and warned that share prices and currencies would come under increasing pressure as the bills mounted.",business,0
1,"Barclays shares up on merger talk Barclays shares up on merger talk Shares in UK banking group Barclays have risen on Monday following a weekend press report that it had held merger talks with US bank Wells Fargo. A tie-up between Barclays and California-based Wells Fargo would create the world's fourth biggest bank, valued at $180bn (£96bn). Barclays has declined to comment on the report in the Sunday Express, saying it does not respond to market speculation. The two banks reportedl

In [11]:
article_id=df.index.tolist()
print(article_id[:5])
categories=df["category_id"]   #storing the category_id value in a list
print(categories[:5])


[0, 1, 2, 3, 4]
0    0
1    0
2    0
3    0
4    0
Name: category_id, dtype: int64


# Tokenizing Text

 For tokenizing process it is important to have article in a list form. Hence article variable stores all the articles of the df dataframe in alist for further pre-processing

In [12]:
article=df["Article"].tolist() #converting article column of data frame to a list for tokenizing it
print("Read %d raw text documents" % len(article))

Read 1408 raw text documents


Raw text documents are textual, not numeric. The first step in analysing unstructured documents is to split the raw text into individual tokens, each corresponding to a single term (word). As an example:

In [13]:
print(article[0])

Asian quake hits European shares Asian quake hits European shares  Shares in Europe's leading reinsurers and travel firms have fallen as the scale of the damage wrought by tsunamis across south Asia has become apparent.  More than 23,000 people have been killed following a massive underwater earthquake and many of the worst hit areas are popular tourist destinations. Reisurance firms such as Swiss Re and Munich Re lost value as investors worried about rebuilding costs. But the disaster has little impact on stock markets in the US and Asia.  Currencies including the Thai baht and Indonesian rupiah weakened as analysts warned that economic growth may slow. "It came at the worst possible time," said Hans Goetti, a Singapore-based fund manager. "The impact on the tourist industry is pretty devastating, especially in Thailand." Travel-related shares dropped in Europe, with companies such as Germany's TUI and Lufthansa and France's Club Mediterranne sliding. Insurers and reinsurance firms we

#### We will use the built-in scikit-learn tokenizer to split this document into tokens. Note that we will perform case conversion first to convert the entire text to lowercase

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
tokenize = CountVectorizer().build_tokenizer()

# convert to lowercase (Normalisation), then tokenise 
tokens = []
for articles in article:
    tokens.append(tokenize(articles.lower()))

In [29]:
#printing sample tokens
print(tokens[0])
print("length:", len(tokens[0]))

['asian', 'quake', 'hits', 'european', 'shares', 'asian', 'quake', 'hits', 'european', 'shares', 'shares', 'in', 'europe', 'leading', 'reinsurers', 'and', 'travel', 'firms', 'have', 'fallen', 'as', 'the', 'scale', 'of', 'the', 'damage', 'wrought', 'by', 'tsunamis', 'across', 'south', 'asia', 'has', 'become', 'apparent', 'more', 'than', '23', '000', 'people', 'have', 'been', 'killed', 'following', 'massive', 'underwater', 'earthquake', 'and', 'many', 'of', 'the', 'worst', 'hit', 'areas', 'are', 'popular', 'tourist', 'destinations', 'reisurance', 'firms', 'such', 'as', 'swiss', 're', 'and', 'munich', 're', 'lost', 'value', 'as', 'investors', 'worried', 'about', 'rebuilding', 'costs', 'but', 'the', 'disaster', 'has', 'little', 'impact', 'on', 'stock', 'markets', 'in', 'the', 'us', 'and', 'asia', 'currencies', 'including', 'the', 'thai', 'baht', 'and', 'indonesian', 'rupiah', 'weakened', 'as', 'analysts', 'warned', 'that', 'economic', 'growth', 'may', 'slow', 'it', 'came', 'at', 'the', 'wo

## STOP Words Removal

#### We immediately see that many of the words here are not useful (e.g. "to", "the" etc.). Scikit-learn provides a list of such stop words:


In [30]:
# load English stop words
from sklearn.feature_extraction import text
stopwords = text.ENGLISH_STOP_WORDS
print(stopwords)


frozenset({'almost', 'couldnt', 'around', 'were', 'anyhow', 'former', 'own', 'there', 'whole', 'namely', 'could', 'seeming', 'hereafter', 'sometime', 'had', 'often', 'my', 'to', 'while', 'down', 'each', 'system', 'whatever', 'who', 'until', 'although', 'them', 'at', 'latter', 'less', 'give', 'through', 'is', 'full', 'nothing', 'or', 'and', 'both', 'interest', 'hereby', 'might', 'move', 'a', 'rather', 'about', 'top', 'fifteen', 'thereby', 'amount', 'should', 'noone', 'therefore', 'throughout', 'we', 'inc', 'six', 'become', 'now', 'may', 'part', 'i', 'moreover', 'cant', 'himself', 'show', 'how', 'herself', 'detail', 'whether', 'above', 'hers', 'against', 'her', 'be', 'can', 'here', 'never', 'this', 'seem', 'him', 'itself', 'side', 'few', 'wherever', 'forty', 'thence', 'enough', 'seems', 'yours', 'such', 'when', 'nine', 'already', 'again', 'mill', 'go', 'sixty', 'between', 'after', 'neither', 'therein', 'among', 'eight', 'call', 'nor', 'either', 'everything', 'below', 'see', 'themselves',

#### We can filter out these stopwords from our document:

In [31]:
for i in range (0, len(tokens)):
    filtered_token = []
    for token in tokens[i]:
        if token not in stopwords:
            filtered_token.append(token)
    tokens[i] = filtered_token

In [32]:
# Example after removing stop words

print(tokens[0])
print("length:", len(tokens[0]))

['asian', 'quake', 'hits', 'european', 'shares', 'asian', 'quake', 'hits', 'european', 'shares', 'shares', 'europe', 'leading', 'reinsurers', 'travel', 'firms', 'fallen', 'scale', 'damage', 'wrought', 'tsunamis', 'south', 'asia', 'apparent', '23', '000', 'people', 'killed', 'following', 'massive', 'underwater', 'earthquake', 'worst', 'hit', 'areas', 'popular', 'tourist', 'destinations', 'reisurance', 'firms', 'swiss', 'munich', 'lost', 'value', 'investors', 'worried', 'rebuilding', 'costs', 'disaster', 'little', 'impact', 'stock', 'markets', 'asia', 'currencies', 'including', 'thai', 'baht', 'indonesian', 'rupiah', 'weakened', 'analysts', 'warned', 'economic', 'growth', 'slow', 'came', 'worst', 'possible', 'time', 'said', 'hans', 'goetti', 'singapore', 'based', 'fund', 'manager', 'impact', 'tourist', 'industry', 'pretty', 'devastating', 'especially', 'thailand', 'travel', 'related', 'shares', 'dropped', 'europe', 'companies', 'germany', 'tui', 'lufthansa', 'france', 'club', 'mediterran

We will repeat this process for all documents

In [33]:
all_filtered_tokens = []
for doc in article:
    # tokenize the next document
    tokens = tokenize(doc.lower())
    # remove the stopwords
    filtered_tokens = []
    for token in tokens:
        if not token in stopwords:
            filtered_tokens.append(token)  
    # add to the overall list
    all_filtered_tokens.append( filtered_tokens )
print("Created %d filtered token lists" % len(all_filtered_tokens) )

Created 1408 filtered token lists


### Counting Tokens

A simple type of analysis that we might do is to count the number of times specific terms (words) appear in our corpus. We could do this by creating a dictionary of term frequency counts

In [38]:
counts = {}
# process filtered tokens for each document
for doc_tokens in all_filtered_tokens:
    for token in doc_tokens:
        # increment existing?
        if token in counts:
            counts[token] += 1
        # a new term?
        else:
            counts[token] = 1
print("Found %d unique terms in this corpus" % len(counts))

Found 22601 unique terms in this corpus


### The top 20 most Frequent terms

In [35]:
import operator
sorted_counts = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
for i in range(20):
    term = sorted_counts[i][0]
    count = sorted_counts[i][1]
    print( "%s (count=%d)" % ( term, count )  )

said (count=4119)
year (count=1563)
new (count=1233)
people (count=1204)
mr (count=1092)
world (count=966)
time (count=934)
game (count=886)
news (count=772)
online (count=733)
just (count=683)
market (count=652)
like (count=618)
games (count=615)
players (count=605)
make (count=604)
company (count=602)
years (count=600)
technology (count=583)
firm (count=555)


# Lemmatisation: WordNetLemmatizer

In [44]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

for i in range (0, len(all_filtered_tokens)):
    filtered_token = []
    for token in all_filtered_tokens[i]:
        filtered_token.append(lemmatizer.lemmatize(token))
    all_filtered_tokens[i] = filtered_token

In [46]:
# Example after lemmatising
#print(article_title[0])
print(all_filtered_tokens[0])
print("length:", len(all_filtered_tokens[0]))

['asian', 'quake', 'hit', 'european', 'share', 'asian', 'quake', 'hit', 'european', 'share', 'share', 'europe', 'leading', 'reinsurers', 'travel', 'firm', 'fallen', 'scale', 'damage', 'wrought', 'tsunami', 'south', 'asia', 'apparent', '23', '000', 'people', 'killed', 'following', 'massive', 'underwater', 'earthquake', 'worst', 'hit', 'area', 'popular', 'tourist', 'destination', 'reisurance', 'firm', 'swiss', 'munich', 'lost', 'value', 'investor', 'worried', 'rebuilding', 'cost', 'disaster', 'little', 'impact', 'stock', 'market', 'asia', 'currency', 'including', 'thai', 'baht', 'indonesian', 'rupiah', 'weakened', 'analyst', 'warned', 'economic', 'growth', 'slow', 'came', 'worst', 'possible', 'time', 'said', 'han', 'goetti', 'singapore', 'based', 'fund', 'manager', 'impact', 'tourist', 'industry', 'pretty', 'devastating', 'especially', 'thailand', 'travel', 'related', 'share', 'dropped', 'europe', 'company', 'germany', 'tui', 'lufthansa', 'france', 'club', 'mediterranne', 'sliding', 'ins

In [47]:
corpus = []
for articles in all_filtered_tokens:
    corpus.append(" ".join(articles))

# example
corpus[0]

'asian quake hit european share asian quake hit european share share europe leading reinsurers travel firm fallen scale damage wrought tsunami south asia apparent 23 000 people killed following massive underwater earthquake worst hit area popular tourist destination reisurance firm swiss munich lost value investor worried rebuilding cost disaster little impact stock market asia currency including thai baht indonesian rupiah weakened analyst warned economic growth slow came worst possible time said han goetti singapore based fund manager impact tourist industry pretty devastating especially thailand travel related share dropped europe company germany tui lufthansa france club mediterranne sliding insurer reinsurance firm pressure europe share munich swiss world biggest reinsurers fell market speculated cost rebuilding asia zurich financial allianz axa suffered decline value loss smaller reflecting market view reinsurers likely pick bulk cost worry size insurance liability dragged europe

# Creating a document-term matrix

### Bag-of-Words Representation¶

In the *bag-of-words model*, each document is represented by a vector in a *m*-dimensional coordinate space, where *m* is number of unique terms across all documents. This set of terms is called the corpus *vocabulary*. Note that the positioning (context) of terms within the original document is lost in this model.

Since each document can be represented as a term vector, we can stack these vectors to create a full *document-term matrix*. We can easily create this matrix from a list of document strings using Scikit-learn:

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.shape)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
print("Number of terms in model is %d" % len(vectorizer.vocabulary_))

In [ ]:
terms = vectorizer.get_feature_names()
vocab = vectorizer.vocabulary_
print("Vocabulary has %d distinct terms" % len(terms))

In [ ]:
df_count = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names(), index = article_id).T
df_count

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus)

In [ ]:
terms = vectorizer.get_feature_names()
vocab = vectorizer.vocabulary_
print("Vocabulary has %d distinct terms" % len(terms))

In [ ]:
print(tfidf.shape)

# convert to dense matrix
tfidf_dense = tfidf.toarray()
df_tfidf = pd.DataFrame(tfidf.toarray(), columns = vectorizer.get_feature_names(), index = article_id).T
df_tfidf

# Classification

## Split the data in training and test set

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test, target_train, target_test = train_test_split(tfidf, categories, test_size=0.2)

### The size of the training set and test set

In [ ]:
print("Training set size is %d" % dataset_train.shape[0] )
print("Test set size is %d" % dataset_test.shape[0] )

## KNN Classifier

k-NN is a type of instance-based learning, or lazy learning, where the function is only approximated locally and all computation is deferred until classification. The k-NN algorithm is among the simplest of all machine learning algorithms.
The neighbors are taken from a set of objects for which the class (for k-NN classification) or the object property value (for k-NN regression) is known. This can be thought of as the training set for the algorithm, though no explicit training step is required.
he training examples are vectors in a multidimensional feature space, each with a class label. The training phase of the algorithm consists only of storing the feature vectors and class labels of the training samples.

In the classification phase, k is a user-defined constant, and an unlabeled vector (a query or test point) is classified by assigning the label which is most frequent among the k training samples nearest to that query point.

A commonly used distance metric for continuous variables is Euclidean distance. For discrete variables, such as for text classification, another metric can be used, such as the overlap metric (or Hamming distance). In the context of gene expression microarray data, for example, k-NN has also been employed with correlation coefficients such as Pearson and Spearman.[3] Often, the classification accuracy of k-NN can be improved significantly if the distance metric is learned with specialized algorithms such as Large Margin Nearest Neighbor or Neighbourhood components analysis.

### Adavatages
-Simple to implement
-Flexible to feature / distance choices
-Naturally handles multi-class cases
-Can do well in practice with enough representative data

### Disadvantages
- Large search problem to find nearest neighbours
- Storage of data
- Must know we have a meaningful distance function

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(dataset_train, target_train) #fitting the model
print(model)

Predcted datalabels of the test dataset

In [ ]:
predicted = model.predict(dataset_test)
predicted  #prediction by the model

In [ ]:
unique_category

In [ ]:
num_b = (predicted == 0).sum()
num_s = (predicted == 1).sum()
num_t = (predicted == 2).sum()

print( "Number of labels predicted as 'business' : %d" % num_b )
print( "Number of labels predicted as 'sport' : %d" % num_s )
print( "Number of labels predicted as 'technology' : %d" % num_t )

Example of Correct label versus predicted label

In [ ]:
print("Predictions\n", predicted[:10])
print("Correct labels\n", target_test[:10])

# The Confusion Matrix of  KNN

In [ ]:
# import all of the scikit-learn evaluation functionality
from sklearn.metrics import *
# build the confusion matrix
cm = confusion_matrix(target_test, predicted)
print(cm)

### The accuracy of the Knn model

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(target_test, predicted)

# Decision Tree Classifier

Decision tree learning uses a decision tree (as a predictive model) to go from observations about an item (represented in the branches) to conclusions about the item's target value (represented in the leaves). It is one of the predictive modelling approaches used in statistics, data mining and machine learning. Tree models where the target variable can take a discrete set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees.


### Decision tree advantages
- Amongst other data mining methods, decision trees have various advantages:

- Simple to understand and interpret. People are able to understand decision tree models after a brief explanation. Trees can also be displayed graphically in a way that is easy for non-experts to interpret.[16]
- Able to handle both numerical and categorical data.[16] Other techniques are usually specialised in analysing datasets that have only one type of variable. (For example, relation rules can be used only with nominal variables while neural networks can be used only with numerical variables or categoricals converted to 0-1 values.)
- Requires little data preparation. Other techniques often require data normalization. Since trees can handle qualitative predictors, there is no need to create dummy variables.[16]
- Uses a white box model. If a given situation is observable in a model the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model, the explanation for the results is typically difficult to understand, for example with an artificial neural network.
- Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
- Non-statistical approach that makes no assumptions of the training data or prediction residuals; e.g., no distributional, independence, or constant variance assumptions
- Performs well with large datasets. Large amounts of data can be analysed using standard computing resources in reasonable time.
- Mirrors human decision making more closely than other approaches.[16] This could be useful when modeling human decisions/behavior.
- Robust against co-linearity, particularly boosting
- In built feature selection. Additional irrelevant feature will be less used so that they can be removed on subsequent runs.


### Limitations
- Trees do not tend to be as accurate as other approaches.
- Trees can be very non-robust. A small change in the training data can result in a big change in the tree, and thus a big change in final predictions.
- The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts Consequently, practical decision-tree learning algorithms are based on heuristics such as the greedy algorithm where locally-optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally-optimal decision tree. To reduce the greedy effect of local-optimality some methods such as the dual information distance (DID) tree were proposed.[19]
- Decision-tree learners can create over-complex trees that do not generalize well from the training data. (This is known as overfitting.[20]) Mechanisms such as pruning are necessary to avoid this problem (with the exception of some algorithms such as the Conditional Inference approach, that does not require pruning.
- There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems. In such cases, the decision tree becomes prohibitively large. Approaches to solve the problem involve either changing the representation of the problem domain (known as propositionalization)[21] or using learning algorithms based on more expressive representations (such as statistical relational learning or inductive logic programming).
- For data including categorical variables with different numbers of levels, information gain in decision trees is biased in favor of those attributes with more levels.[22] However, the issue of biased predictor selection is avoided by the Conditional Inference approach[12], a two-stage approach[23], or adaptive leave-one-out feature selection.

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(dataset_train, target_train) #fitting the model

In [ ]:
predicted = clf.predict(dataset_test)
predicted  #prediction by the model

In [ ]:
num_b = (predicted == 0).sum()
num_s = (predicted == 1).sum()
num_t = (predicted == 2).sum()

print( "Number of labels predicted as 'business' : %d" % num_b )
print( "Number of labels predicted as 'sport' : %d" % num_s )
print( "Number of labels predicted as 'technology' : %d" % num_t )

In [ ]:
print("Predictions\n", predicted[:10])
print("Correct labels\n", target_test[:10])

### The accuracy of the Decision tree model

In [ ]:
from sklearn.metrics import accuracy_score
score1=accuracy_score(target_test, predicted)

# The Confusion Matrix of Decision Tree

In [ ]:
# import all of the scikit-learn evaluation functionality
from sklearn.metrics import *
# build the confusion matrix
cm = confusion_matrix(target_test, predicted)
print(cm)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Plotting the graph

In [ ]:
prediction_accuracy=[score,score1]
y_pos = np.arange(len(prediction_accuracy))
freq_series = pd.Series.from_array(prediction_accuracy)
x_labels = list(y_pos)
z = ['KNN(N=3)', 'Decision Tree']
# Plot the figure.
plt.figure(figsize=(10, 6))
ax = freq_series.plot(kind='bar')
ax.set_title('Accuracy of classifiers')
ax.set_xlabel('Classifiers')
ax.set_ylabel('Accuracy Percentage')
ax.set_xticklabels(list(z))
rects = ax.patches
# putting labels.
labels =[i for i in list(prediction_accuracy)]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width(), height, label, ha='center', va='bottom')

# Cross Validation

A problem with simply randomly splitting a dataset into two sets is that each random split might give different results. We are also ignoring a portion of your dataset. One way to address this is to use *k-fold cross-validation* to evaluate a classifier:
1. Divide the data into k disjoint subsets - “folds” (e.g. k=5).
2. For each of k experiments, use k-1 folds for training and the selected one fold for testing.
3. Repeat for all k folds, average the accuracy/error rates.

While this is a relatively complex process, scikit-learn allows us to achieve this using a single command. Let's do a 2-fold cross-validation of the KNN classifier

In [ ]:
# create a single classifier
model = KNeighborsClassifier(n_neighbors=3)
# apply 2-fold cross-validation, measuring accuracy each time
from sklearn.model_selection import cross_val_score
acc_scores = cross_val_score(model, dataset_train, target_train, cv=10, scoring="accuracy")
print(acc_scores)

In [ ]:
print("KNN: Mean cross-validation accuracy = %.2f" % acc_scores.mean() )

In [ ]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
acc_scores1 = cross_val_score(model, dataset_train, target_train, cv=10, scoring="accuracy")
print("The Decision tree average cross-validation  = %.2f" % acc_scores1.mean() )

# Plotting the graph

In [ ]:
prediction_accuracy=[acc_scores.mean(),acc_scores1.mean()]
y_pos = np.arange(len(prediction_accuracy))
freq_series = pd.Series.from_array(prediction_accuracy)
x_labels = list(y_pos)
z = ['KNN(N=3)', 'Decision Tree']
# Plot the figure.
plt.figure(figsize=(10, 6))
ax = freq_series.plot(kind='bar')
ax.set_title('Accuracy of classifiers after kfold validation')
ax.set_xlabel('Classifiers')
ax.set_ylabel('Accuracy Percentage')
ax.set_xticklabels(list(z))
rects = ax.patches
# putting labels.
labels =[i for i in list(prediction_accuracy)]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width(), height, label, ha='center', va='bottom')